## 6.3 D Code; EM Algorithm for Noisy-OR

In [1]:
import numpy as np
import itertools
import time
import math

pi = [.05]*23
X_data = np.loadtxt('./spectX.txt',dtype = float)
Y_data = np.loadtxt('./spectY.txt',dtype = float)
T = Y_data.shape[0]
num_inputs = 23
num_iter   = 265

In [15]:
def likelihood(p_s,X,Y,T):
    
    L = 0
    for t in range(T): # iterate over all of the entries
        pre_log = 0
        multiple = 1
        if Y[t] == 1:
            for i in range(num_inputs): 
                multiple = multiple*((1-p_s[i])**(X[t,i]))
            pre_log = 1-multiple
            
        else: # Y[t] == 0
            for i in range(num_inputs):
                multiple = multiple*((1-p_s[i])**(X[t,i]))
            pre_log = multiple
            
        L += math.log(pre_log)
    
    return L/T

In [5]:
def calc_mistakes(X,Y,pi,T):
    
    num_mistakes = 0
    for t in range(T): # num entries
        #calculate the probability of Y = (1-multiple(1-pi)^xi)
        multiplier = 1
        # caclculate the iterable product
        for i in range(num_inputs): # num inputs
            multiplier = multiplier * ((1-pi[i])**X[t,i]) # (1-pi)^(x_it)
            
        Y_pred = np.round(1 - multiplier,0) # rounds it up or down to form the prediction
        if Y_pred != Y[t]:
            num_mistakes += 1
            
    return num_mistakes
       

In [6]:
def calc_pi_update(X,Y,pi,T,num_inputs):
    

    # preliminarily calculate the denominator of the expectation: 1 - PI{ (1-pi)**xi } for each of the T samples
    multiplier = np.ones((T,1))
    E_bottom   = np.zeros((T,1))
    for t in range(T):
        for j in range(num_inputs): 
            multiplier[t] = multiplier[t] * ((1-pi[j])**X[t,j]) # (1-pi)^(x_it)  
                
        E_bottom[t] = 1 - multiplier[t] # 1 - PI{ (1-pi)**xi }
        
    # calculate the expectation and the parameter update for p_i
    pi_new = np.zeros((num_inputs,1))
    for i in range(num_inputs):
        count_Ti = np.count_nonzero(X[:,i]==1) # num times X_i = 1
        E_Sum    = 0                           # sum of the expectation over T entries
        
        for t in range(T):
            # calculate numberator: y_i*x_i*p_i
            E_top = Y[t]*X[t,i]*pi[i]
            # calculate total expectation
            E_Sum += E_top/E_bottom[t]
        
        pi_new[i] = E_Sum/count_Ti
        
    return pi_new      
         
    

In [13]:
def main_func(X,Y,pi,T,n_iter,num_in):
    
    '''
    X = x data
    Y = y data
    pi = values of p_i
    T  = total # of samples
    n_iter = number of iterations of the algo
    num_in = number of inputs to the noisy-or
    
    '''
    print_list = [0,1,2,4,8,16,32,64,128,256]
    for i in range(n_iter+1): # number of iterations of the EM algorithm
        L = likelihood(pi,X,Y,T)
        num_mist = calc_mistakes(X,Y,pi,T)
        if i in print_list:
            print('Iteration: %i , Likelihood: %.4f , num mist: %i' % (i,L,num_mist))
        
        pi = calc_pi_update(X,Y,pi,T,num_inputs)    
        

In [14]:
main_func(X_data,Y_data,pi,T,num_iter,num_inputs)

Iteration: 0 , Likelihood: -0.9581 , num mist: 175
Iteration: 1 , Likelihood: -0.4959 , num mist: 56
Iteration: 2 , Likelihood: -0.4082 , num mist: 43
Iteration: 4 , Likelihood: -0.3646 , num mist: 42
Iteration: 8 , Likelihood: -0.3475 , num mist: 44
Iteration: 16 , Likelihood: -0.3346 , num mist: 40
Iteration: 32 , Likelihood: -0.3226 , num mist: 37
Iteration: 64 , Likelihood: -0.3148 , num mist: 37
Iteration: 128 , Likelihood: -0.3112 , num mist: 36
Iteration: 256 , Likelihood: -0.3102 , num mist: 36
